# 6.3  다나와 크롤링 데이터 전처리

### 6.3.1 다나와 크롤링 데이터 불러오기 

In [4]:
# 예제 6-17 다나와 크롤링 결과 가져오기 
import pandas as pd

data = pd.read_excel('./files/3_1_danawa_crawling_result.xlsx')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품명     400 non-null    object
 1   스펙 목록   400 non-null    object
 2   가격      420 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 10.0+ KB


,상품명,스펙 목록,가격
0,LG전자 오브제컬렉션 코드제로 A9S AX9884,핸디스틱청소기 / \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,855210
1,삼성전자 비스포크 제트 VS20B956AX,핸디스틱청소기 / \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,450370
2,샤오미 미홀 M22,핸디스틱청소기 / \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,154220
3,LG전자 오브제컬렉션 코드제로 A9 AS9202WD,핸디스틱청소기 / \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,453310
4,LG전자 오브제컬렉션 코드제로 A9S AX9984,핸디스틱청소기 / \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,992930


### 6.3.2 회사명, 모델명 정리 

In [5]:
# NaN 값을 공백 문자열로 대체
data['상품명'].fillna('', inplace=True)
# 위 내용을 추가한 이유는 NaN일경우 NaN은 실제로 부동 소수점(float)으로 간주되며, 따라서 'split' 메서드를 사용할 수 없기 때문
# fillna 메서드는 DataFrame에서 결측값을 원하는 값으로 변경하는 메서드입니다.


#예제 6-18 회사명 + 모델명 분리
company_list = []
product_list = []
for title in data['상품명']:
    title_info = title.split(' ', 1)
    company_name = title_info[0]
    product_name = title_info[1] if len(title_info) > 1 else ''  # title_info의 길이가 1보다 크면 product_name으로 할당
    company_list.append(company_name)
    product_list.append(product_name)


### 6.3.4 스펙 목록에서 카테고리, 사용시간, 흡입력을 추출해서 정리


In [6]:
# 예제 6-19 첫 번째 제품의 스펙 목록 분리
spec_list = [spec.replace('\t', '') for spec in data['스펙 목록'][0].split(' / ')]
print(spec_list)


['핸디스틱청소기', '무선', '흡입+물걸레(동시)', '흡입력: 250W', '소비전력: 620W', '2023년형', '[구성] 거치대: 올인원타워', '먼지비움', '충전', 'UVC LED', '액서서리수납', '스탠드거치', '먼지비움시간: 30초', '브러쉬: 와이드바닥', '물걸레: 스팀, 고온, 일반', '솔형', '틈새', '먼지봉투: 2.5L', '[배터리] 사용시간: 30분(최대)', '충전시간: 4시간', '분리형(2개)', '리튬이온', '25.2V', '[청소] 싸이클론흡입', 'LED라이트', '자동물공급', '스마트인버터모터', '[부가] 스마트폰제어', '대화형알림창', '워셔블헤파필터', '5단계여과', '색상: 카밍베이지', '무게: 2.47kg', '크기(가로x세로x깊이): 300x1120x245mm']


In [7]:
# 예제 6-20 카테고리 정보 추출
category = spec_list[0] 
category 

'핸디스틱청소기'

In [8]:
# 예제 6-21 ‘사용시간’, ‘흡입력’이 포함된 원소 추출 
for spec in spec_list:
    if '사용시간' in spec: 
         use_time_spec = spec 
    elif '흡입력' in spec:
        suction_spec = spec  
print(use_time_spec)
print(suction_spec)

[배터리] 사용시간: 30분(최대)
흡입력: 250W


In [9]:
# 예제 6-22 정량적인 수치 추출 
use_time_value = use_time_spec.split(' ')[1].strip() 
suction_value = suction_spec.split(' ')[1].strip()
print(use_time_value)
print(suction_value)

사용시간:
250W


In [10]:
# NaN 값을 빈 문자열로 대체
data['스펙 목록'].fillna('', inplace=True)

# 예제 6-23 카테고리, 사용시간, 흡입력 추출 
category_list = []
use_time_list = []
suction_list = [] 

for spec_data in data['스펙 목록']:
    # ' / ' 기준으로 스펙 분리하기 
    spec_list = spec_data.split(' / ')
    
    # 카테고리 추출하기
    category = spec_list[0] 
    category_list.append(category)
    
    # 사용시간, 흡입력 추출 
    ## 사용시간, 흡입력 정보가 없는 제품을 위해 변수를 생성 
    use_time_value = None 
    suction_value = None 
    
    ## spec_list의 각 원소에서 사용시간, 흡입력 수치 추출
    for spec in spec_list:
        if '사용시간' in spec: 
            use_time_value = spec.split(' ')[1].strip()             
        if '흡입력' in spec:
            suction_value = spec.split(' ')[1].strip() 
    use_time_list.append(use_time_value)
    suction_list.append(suction_value)

In [11]:
# 예제 6-24 카테고리, 사용시간, 흡입력에 대한 전처리 결과 확인
print("카테고리", len(category_list), category_list[0:5])
print("사용시간", len(use_time_list), use_time_list[0:5])
print("흡입력", len(suction_value), suction_value[0:5])

카테고리 420 ['핸디스틱청소기', '핸디스틱청소기', '핸디스틱청소기', '핸디스틱청소기', '핸디스틱청소기']
사용시간 420 ['사용시간:', '사용시간:', None, '사용시간:', '사용시간:']
흡입력 7 41000


### 6.3.5 무선청소기 사용시간 단위 통일시키기

In [12]:
# 예제 6-25 사용시간을 분 단위로 조정하는 함수
def convert_time_minute(time):
    try:
        if '시간' in time:
            hour = time.split('시간')[0]
            if '분' in time:
                minute = time.split('시간')[-1].split('분')[0]
                # 마지막 멘트에 '분'이라는 글자가 있으면
            else:
                minute = 0
        else: 
            hour = 0
            minute = time.split('분')[0]
        return int(hour)*60 + int(minute)
    except:
        return None                

In [13]:
# 예제 6-26 사용시간을 분 단위로 조정하는 함수의 테스트
times = ["40분", "4분", "1시간", "3시간30분", "4시간"]
for time in times:
    time_value = convert_time_minute(time)
    print(time, "=", time_value)

40분 = 40
4분 = 4
1시간 = 60
3시간30분 = 210
4시간 = 240


In [14]:
# 예제 6-27 모델별 사용시간을 분 단위로 통일하기
new_use_time_list = []
for time in use_time_list:
    value = convert_time_minute(time)
    new_use_time_list.append(value)

### 6.3.6 무선 청소기 흡입력 단위 통일시키기

In [15]:
# 예제 6-28 흡입력 단위를 통일시키는 함수
# AW, W, PA는 흡입력 단위 | 1W = 1AW = 100PA 따라서 PA만 100으로 나눈겨
def get_suction(value):
    try:
        value = value.upper()
        if "AW" in value or "W" in value:
            result = value.replace("A", "").replace("W","")
            result = int(result.replace(",",""))
            # 인트로 변환후 글자 삭제.
        elif "PA" in value:
            result = value.replace("PA","")
            result = int(result.replace(",",""))/100
            # 인트로 변환 + 100으로 나누기 + 글자 삭제
        else:
            result = None
        return result
    except:
        return None

In [18]:
# 예제 6-29 흡입력 단위 통일시키기
new_suction_list = []
for power in suction_list:
    value = get_suction (power)
    new_suction_list.append(value)

### 6.3.7 다나와 전처리 결과를 엑셀로 저장


In [17]:
# 예제 6-30 전처리 데이터 확인
pd_data = pd.DataFrame()
pd_data['카테고리'] = category_list
pd_data['회사명'] = company_list
pd_data['제품'] = product_list
pd_data['가격'] = data['가격']
pd_data['사용시간'] = new_use_time_list
pd_data['흡입력'] = new_suction_list
pd_data.head()

,카테고리,회사명,제품,가격,사용시간,흡입력
0,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9S AX9884,855210,NaN,250.0
1,핸디스틱청소기,삼성전자,비스포크 제트 VS20B956AX,450370,NaN,220.0
2,핸디스틱청소기,샤오미,미홀 M22,154220,NaN,NaN
3,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9 AS9202WD,453310,NaN,210.0
4,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9S AX9984,992930,NaN,250.0


In [21]:
# 예제 6-31 카테고리 분류 기준 및 데이터 개수 점검
pd_data['카테고리'].value_counts() 

카테고리
핸디스틱청소기            352
                    20
욕실청소기               17
핸디스틱청소기+로봇청소기       15
스틱청소기                8
탁상용청소기               4
창문청소기                2
핸디스틱청소기+공기청정기세트      1
신발세척기                1
Name: count, dtype: int64

In [23]:
# 예제 6-32 핸디/스틱청소기만 선택
pd_data_final = pd_data[pd_data['카테고리'].isin(['핸디스틱청소기'])]
len(pd_data_final)

352

In [25]:
# 예제 6-33 엑셀로 저장
pd_data_final.to_excel('./files/3_2_danawa_data_final.xlsx', index = False)